In [1]:
import pandas as pd
from src.student_eval.app.evaluation import models
df = pd.DataFrame({
    'user_id'   : ['alice','alice','bob','bob']*3,
    'skill_name': ['fractions']*6 + ['equations']*6,
    'correct'   : [0,1,0,0,  1,1,  1,0,0,1,1,1],
    'item_id'   : ['A','B','A','B']*3
})
df['order_id'] = df.groupby(['user_id','skill_name']).cumcount() + 1


In [2]:
user_id = df['user_id'].to_list()
skill_name = df['skill_name'].to_list()
correct = df['correct'].to_list()
item_id = df['item_id'].to_list()
order_id = df['order_id'].to_list()

model=models.StudentModel()
model.csv_path = "data.csv"
model.model_path = "model.pkl"
results = model.update_dataset(order_id, user_id, skill_name, correct, item_id)
print(results["students_states"])
print(results["skills_states"])

   user_id skill_name  correct item_id
0    alice  fractions        0       A
1    alice  fractions        1       B
2      bob  fractions        0       A
3      bob  fractions        0       B
4    alice  fractions        1       A
5    alice  fractions        1       B
6      bob  equations        1       A
7      bob  equations        0       B
8    alice  equations        0       A
9    alice  equations        1       B
10     bob  equations        1       A
11     bob  equations        1       B
  user_id  equations_learn  fractions_learn  equations_forget  \
1   alice          0.18204          0.14653           0.00000   
2     bob          0.73968          0.01945           0.00000   

   fractions_forget  
1           0.00000  
2           0.00000  
       skill   prior  learns  guesses   slips  forgets
0  equations 0.00000 0.63193  0.43978 0.10387  0.64113
1  fractions 0.00000 0.48813  0.02845 0.08062  0.00656


In [3]:
# order_id = [4, 5, 6]
# skill_name = ["fractions", "fractions", "fractions"]
# user_id = ["alice", "alice", "alice"]
# item_id = ["A", "B", "A"]
# correct = [1, 0, 1]
results = model.update_dataset(order_id, user_id, skill_name, correct, item_id)
print(results["students_states"])
print(results["skills_states"])

   user_id skill_name  correct item_id
0    alice  fractions        0       A
1    alice  fractions        1       B
2      bob  fractions        0       A
3      bob  fractions        0       B
4    alice  fractions        1       A
5    alice  fractions        1       B
6      bob  equations        1       A
7      bob  equations        0       B
8    alice  equations        0       A
9    alice  equations        1       B
10     bob  equations        1       A
11     bob  equations        1       B
                            value
skill     param   class          
fractions prior   default 0.00000
          learns  1       0.07160
                  2       1.00000
                  Default 0.50473
          guesses default 0.00087
          slips   default 0.06689
          forgets 1       0.00000
                  2       0.00000
                  Default 0.00000
equations prior   default 0.00000
          learns  1       0.97836
                  2       0.99757
                 

In [12]:
import pandas as pd
from pyBKT.models import Model

# ──────────────────────────────────────────
# 1) Primer lote de interacciones
# ──────────────────────────────────────────
df_1 = pd.DataFrame({
    'user_id'   : ['alice', 'alice', 'bob', 'bob'],
    'skill_name': ['fractions']*4,
    'correct'   : [0, 1, 0, 0],
    'order_id'  : [1, 2, 1, 2]
})

# Entrenamos un modelo básico: prior por alumno (multiprior)
model = Model(seed=0)
model.fit(
    data=df_1,
    skills='fractions',
    # multiprior ='user_id',      # prior por alumno
    multilearn ='user_id',      # learns / forgets por alumno
    defaults   ={'multilearn':'user_id', 'multiprior':'user_id'}
)


print("— Parámetros tras el primer ajuste —")
print(model.params().loc['fractions'])

# ──────────────────────────────────────────
# 2) Segundo lote: llegan nuevas respuestas
# ──────────────────────────────────────────
df_2 = pd.DataFrame({
    'user_id'   : ['alice', 'bob', 'carol', 'carol'],
    'skill_name': ['fractions']*4,
    'correct'   : [1, 1, 0, 1],
    'order_id'  : [3, 3, 1, 2]   # numeración independiente
})


model.partial_fit(
    data=df_2,
    skills='fractions',
    # multiprior ='user_id',
    multilearn ='user_id',
    defaults   ={'multilearn':'user_id', 'multiprior':'user_id'}
)

print("\n— Parámetros tras partial_fit (lote 1 + lote 2) —")
print(model.params().loc['fractions'])


— Parámetros tras el primer ajuste —
                  value
param   class          
prior   default 0.10932
learns  alice   0.81971
        bob     0.32135
guesses default 0.05949
slips   default 0.40836
forgets alice   0.00000
        bob     0.00000


ValueError: invalid number of learns in initialization